In [1]:
import json
from openai import AzureOpenAI
import time
from IPython.display import clear_output

In [2]:
# あるスレッドに対してリクエストを送信した際のレスポンスを取得
def get_response(client: AzureOpenAI, assistant, thread, request):
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=request
    )

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    start_time = time.time()

    status = run.status

    while status not in ["completed", "cancelled", "expired", "failed"]:
        time.sleep(5)
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
        status = run.status
        print(f'Status: {status}')
        clear_output(wait=True)

    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )

    print(f'Status: {status}')
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    print(messages.model_dump_json(indent=2))

In [3]:
import yaml

file_path = "credentials.yaml"

with open(file_path, "r") as file:
    credentials = yaml.load(file, Loader=yaml.FullLoader)

In [4]:
client = AzureOpenAI(
    api_key=credentials["openai_gpt4"]["key"],
    api_version=credentials["openai_gpt4"]["version"],
    azure_endpoint=credentials["openai_gpt4"]["endpoint"]
)

##### タスク1： 連立方程式を解く

In [3]:
request = "レンガ1個の重さはレンガ1個の30%にGPU5個を加えた重さだ。1トンには100個のレンガと200個のGPUがあるが、レンガ1個の重さは？"

In [ ]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model"
)

In [6]:
thread = client.beta.threads.create()

Thread(id='thread_jB8E2MnqO4cDEUbejJVTq4nq', created_at=1708404653, metadata={}, object='thread')


In [11]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 0 minutes 47 seconds
{
  "data": [
    {
      "id": "msg_9g6KcPBhNRSGXMZCY9WVIqUc",
      "assistant_id": "asst_ScnkLIbPQTyQAZZCwjimhg7n",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "レンガ1個の重さは約7.8125キログラムです。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708405089,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_FqqCBDxBkHAhGLiu8G3Nbo31",
      "thread_id": "thread_jB8E2MnqO4cDEUbejJVTq4nq"
    },
    {
      "id": "msg_lQCw8D3nMYaUpWigjedwxwKC",
      "assistant_id": "asst_ScnkLIbPQTyQAZZCwjimhg7n",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "レンガ1個の重さを \\( R \\) キログラムとし、GPU1個の重さを \\( G \\) キログラムとしましょう。問題の文章によれば、次の等式が成り立ちます：\n\n\\[\nR = 0.3R + 5G\n\\]\n\nこれは、レンガ1個の重さが、その30%にGPU5個の重さを加えたものに等しいという意味です。しかし、これだけでは2つの未知数に対して1つの式しかないため、解

In [ ]:
get_response(client=client, assistant=assistant, thread=thread, request="使用したコードを教えてください")

In [16]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

{
  "data": [
    {
      "id": "msg_w3BXCqBvnvngxLQN5azq2qTy",
      "assistant_id": "asst_ScnkLIbPQTyQAZZCwjimhg7n",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下はレンガ1個の重さを解くために使用したPythonコードと、それに使用された `sympy` モジュールのコードです：\n\n```python\nfrom sympy import symbols, Eq, solve\n\n# 未知数を定義する\nR, G = symbols('R G')\n\n# 与えられた方程式を定義する\neq1 = Eq(R, 0.3*R + 5*G)  # レンガ1個の重さはその30% + GPU5個の重さ\neq2 = Eq(100*R + 200*G, 1000)  # 1トンには100個のレンガと200個のGPUがある\n\n# 方程式を解く\nsolutions = solve((eq1, eq2), (R, G))\n\n# レンガ1個の重さを取得する\nbrick_weight = solutions[R]\nbrick_weight.evalf()\n```\n\nコードではまず `sympy` モジュールから `symbols`, `Eq`, `solve` 関数をインポートします。`symbols` 関数を使って二つの未知数 \\( R \\) と \\( G \\) を定義し、`Eq` 関数を使って二つの方程式 `eq1` と `eq2` を設定します。そして `solve` 関数を使ってこれらの方程式を同時に解き、`R` (レンガ1個の重さ) を求めます。`evalf()` メソッドは数値解を求めるために使われます。\n\n実際に実行されたコードにより、レンガ1個の重さは約7.8125キログラムであると求めることができました。"
          },
          "type": "text"
        }
      ],
      "cre

In [17]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

もちろんです。以下はレンガ1個の重さを解くために使用したPythonコードと、それに使用された `sympy` モジュールのコードです：

```python
from sympy import symbols, Eq, solve

# 未知数を定義する
R, G = symbols('R G')

# 与えられた方程式を定義する
eq1 = Eq(R, 0.3*R + 5*G)  # レンガ1個の重さはその30% + GPU5個の重さ
eq2 = Eq(100*R + 200*G, 1000)  # 1トンには100個のレンガと200個のGPUがある

# 方程式を解く
solutions = solve((eq1, eq2), (R, G))

# レンガ1個の重さを取得する
brick_weight = solutions[R]
brick_weight.evalf()
```

コードではまず `sympy` モジュールから `symbols`, `Eq`, `solve` 関数をインポートします。`symbols` 関数を使って二つの未知数 \( R \) と \( G \) を定義し、`Eq` 関数を使って二つの方程式 `eq1` と `eq2` を設定します。そして `solve` 関数を使ってこれらの方程式を同時に解き、`R` (レンガ1個の重さ) を求めます。`evalf()` メソッドは数値解を求めるために使われます。

実際に実行されたコードにより、レンガ1個の重さは約7.8125キログラムであると求めることができました。


##### タスク2：株価情報の入った CSV から時系列グラフをプロットする

In [23]:
request = "添付データを元に、Openの時系列グラフを表示してください"

In [18]:
# Upload a file with an "assistants" purpose
file = client.files.create(
    file=open("Microsoft_Stock.csv", "rb"),
    purpose='assistants'
)

In [20]:
# Create an assistant using the file ID
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model",
    file_ids=[file.id]
)

In [22]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_oNSndcFzuMR7ULMibYv1v8u4', created_at=1708405947, metadata={}, object='thread')


In [ ]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

In [29]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_note1yWzmpJZBsJuAWNUpYrZ",
      "assistant_id": "asst_hQfw4uPXYt7eN9TQkOTrx62H",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-m4C29GGD6hFcOqIHfW2eK1ld"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "時系列グラフを表示しました。グラフには、読み込んだデータセットに含まれる'Open'（開始価格）に関するポイントごとにプロットされています。日付がx軸に、'Open'の価格がy軸に表示されています。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708406060,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_vaNNYFQ3D7qZQ4e6s92bhBqx",
      "thread_id": "thread_oNSndcFzuMR7ULMibYv1v8u4"
    },
    {
      "id": "msg_tGmWJ72MGxTwQOcgUczx5E24",
      "assistant_id": "asst_hQfw4uPXYt7eN9TQkOTrx62H",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "CSVフォーマットでファイルを読み込

In [30]:
data = json.loads(messages.model_dump_json(indent=2))
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)

assistant-m4C29GGD6hFcOqIHfW2eK1ld


In [31]:
content = client.files.content(image_file_id)

image = content.write_to_file("image.png")

In [32]:
from PIL import Image

# Display the image in the default image viewer
image = Image.open("image.png")
image.show()

In [35]:
get_response(client=client, assistant=assistant, thread=thread, request="使用したコードを教えてください")

Status: completed
Elapsed time: 0 minutes 23 seconds
{
  "data": [
    {
      "id": "msg_6Fd93qksMSt2Xjv7pIdG0TYn",
      "assistant_id": "asst_hQfw4uPXYt7eN9TQkOTrx62H",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下がファイルを読み込み、'Open'の時系列グラフを表示するために使用したコードです。\n\n```python\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# ファイルを読み込む\nfilename = '/mnt/data/assistant-IJWmhAdARrX6onv5shvF4bMg'\ndf = pd.read_csv(filename)\n\n# 'Date' 列を datetime に変換する\ndf['Date'] = pd.to_datetime(df['Date'])\n\n# 'Open' 時系列グラフをプロットする\nplt.figure(figsize=(14,7))\nplt.plot(df['Date'], df['Open'], marker='o')\nplt.title('Open Price Time Series')\nplt.xlabel('Date')\nplt.ylabel('Open Price')\nplt.grid(True)\nplt.xticks(rotation=45)  # x軸のラベルを見やすくするために回転させる\nplt.tight_layout()  # ラベルが切れないようにする\nplt.show()\n```\n\nこのコードは、CSVファイルを`pandas`を使って読み込み、`matplotlib`を使って時系列データをグラフとしてプロットするために使用されています。プロットには、データフレームの'Date'列がx軸に、'Open'列がy軸に使用されており、各ポイ

In [36]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

{
  "data": [
    {
      "id": "msg_6Fd93qksMSt2Xjv7pIdG0TYn",
      "assistant_id": "asst_hQfw4uPXYt7eN9TQkOTrx62H",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下がファイルを読み込み、'Open'の時系列グラフを表示するために使用したコードです。\n\n```python\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# ファイルを読み込む\nfilename = '/mnt/data/assistant-IJWmhAdARrX6onv5shvF4bMg'\ndf = pd.read_csv(filename)\n\n# 'Date' 列を datetime に変換する\ndf['Date'] = pd.to_datetime(df['Date'])\n\n# 'Open' 時系列グラフをプロットする\nplt.figure(figsize=(14,7))\nplt.plot(df['Date'], df['Open'], marker='o')\nplt.title('Open Price Time Series')\nplt.xlabel('Date')\nplt.ylabel('Open Price')\nplt.grid(True)\nplt.xticks(rotation=45)  # x軸のラベルを見やすくするために回転させる\nplt.tight_layout()  # ラベルが切れないようにする\nplt.show()\n```\n\nこのコードは、CSVファイルを`pandas`を使って読み込み、`matplotlib`を使って時系列データをグラフとしてプロットするために使用されています。プロットには、データフレームの'Date'列がx軸に、'Open'列がy軸に使用されており、各ポイントはマーカーで強調されています。また、x軸の日付のラベルは45度傾けて表示されています。"
      

In [37]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

もちろんです。以下がファイルを読み込み、'Open'の時系列グラフを表示するために使用したコードです。

```python
import pandas as pd
import matplotlib.pyplot as plt

# ファイルを読み込む
filename = '/mnt/data/assistant-IJWmhAdARrX6onv5shvF4bMg'
df = pd.read_csv(filename)

# 'Date' 列を datetime に変換する
df['Date'] = pd.to_datetime(df['Date'])

# 'Open' 時系列グラフをプロットする
plt.figure(figsize=(14,7))
plt.plot(df['Date'], df['Open'], marker='o')
plt.title('Open Price Time Series')
plt.xlabel('Date')
plt.ylabel('Open Price')
plt.grid(True)
plt.xticks(rotation=45)  # x軸のラベルを見やすくするために回転させる
plt.tight_layout()  # ラベルが切れないようにする
plt.show()
```

このコードは、CSVファイルを`pandas`を使って読み込み、`matplotlib`を使って時系列データをグラフとしてプロットするために使用されています。プロットには、データフレームの'Date'列がx軸に、'Open'列がy軸に使用されており、各ポイントはマーカーで強調されています。また、x軸の日付のラベルは45度傾けて表示されています。


##### タスク3：CSV を JSON に変換する

In [38]:
request = """
アップロードしたCSVを次のようなJSONに変換して出力してください
# 出力JSONの例
[
  {"Date": "2015-4-1", "Open": 40.6},
  {"Date": "2015-4-2", "Open": 40.66},
  …
]
"""

In [43]:
file = client.files.create(
    file=open("Microsoft_Stock.csv", "rb"),
    purpose='assistants'
)

In [44]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model",
    file_ids=[file.id]
)

In [46]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_UIkCY7knDNhc6WYGDizqWHsP', created_at=1708407509, metadata={}, object='thread')


In [51]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 2 minutes 12 seconds
{
  "data": [
    {
      "id": "msg_iNLdlzSUrRNE01AKCrdUbtHN",
      "assistant_id": "asst_cIlI2FjsDcUAQu6lZD50WZYx",
      "content": [
        {
          "text": {
            "annotations": [
              {
                "end_index": 128,
                "file_path": {
                  "file_id": "assistant-52uTDvBdratFiglFoRx2cRzG"
                },
                "start_index": 95,
                "text": "sandbox:/mnt/data/stock_data.json",
                "type": "file_path"
              }
            ],
            "value": "JSON形式に変換されたデータはファイル `stock_data.json` に保存されました。以下のリンクからダウンロードできます。\n\n[stock_data.json をダウンロード](sandbox:/mnt/data/stock_data.json)\n\nご質問や他のリクエストがありましたら、お知らせください。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708408140,
      "file_ids": [
        "assistant-52uTDvBdratFiglFoRx2cRzG"
      ],
      "metadata": {},
      "object": "thread.message",
      "role": "

In [52]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_iNLdlzSUrRNE01AKCrdUbtHN",
      "assistant_id": "asst_cIlI2FjsDcUAQu6lZD50WZYx",
      "content": [
        {
          "text": {
            "annotations": [
              {
                "end_index": 128,
                "file_path": {
                  "file_id": "assistant-52uTDvBdratFiglFoRx2cRzG"
                },
                "start_index": 95,
                "text": "sandbox:/mnt/data/stock_data.json",
                "type": "file_path"
              }
            ],
            "value": "JSON形式に変換されたデータはファイル `stock_data.json` に保存されました。以下のリンクからダウンロードできます。\n\n[stock_data.json をダウンロード](sandbox:/mnt/data/stock_data.json)\n\nご質問や他のリクエストがありましたら、お知らせください。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708408140,
      "file_ids": [
        "assistant-52uTDvBdratFiglFoRx2cRzG"
      ],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_pUsrq6qKVkLlrpjUlE3V

In [70]:
data = json.loads(messages.model_dump_json(indent=2))
file_id = data["data"][0]["content"][0]["text"]["annotations"][0]["file_path"]["file_id"]

In [71]:
content = client.files.content(file_id)
content_bytes = content.read()

with open("./task3.json", "wb") as file:
    file.write(content_bytes)

In [74]:
get_response(client=client, assistant=assistant, thread=thread, request="使用したコードを教えてください")

Status: completed
Elapsed time: 0 minutes 42 seconds
{
  "data": [
    {
      "id": "msg_kzeRDufnnGKTIfXVrgs4YHMX",
      "assistant_id": "asst_cIlI2FjsDcUAQu6lZD50WZYx",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下に、アップロードされたCSVデータを読み込み、JSON形式に変換し、ファイルに保存するために使用したPythonコードを記載します。\n\n```python\nimport pandas as pd\nimport json\n\n# Load the uploaded CSV file\nfile_path = '/mnt/data/assistant-DffPxNcC2M77mw9W3cCHbUHy'\ndf = pd.read_csv(file_path)\n\n# We only need the 'Date' and 'Open' columns for the JSON output\ndf_json = df[['Date', 'Open']]\n\n# Convert the 'Date' format to 'YYYY-MM-DD' without time\ndf_json['Date'] = pd.to_datetime(df_json['Date']).dt.strftime('%Y-%m-%d')\n\n# Convert the DataFrame to a list of dictionaries\njson_output = df_json.to_dict(orient='records')\n\n# Convert list of dictionaries to JSON format\njson_data = json.dumps(json_output, ensure_ascii=False, indent=2)\n\n# Save the JSON data to a 

In [75]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_kzeRDufnnGKTIfXVrgs4YHMX",
      "assistant_id": "asst_cIlI2FjsDcUAQu6lZD50WZYx",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下に、アップロードされたCSVデータを読み込み、JSON形式に変換し、ファイルに保存するために使用したPythonコードを記載します。\n\n```python\nimport pandas as pd\nimport json\n\n# Load the uploaded CSV file\nfile_path = '/mnt/data/assistant-DffPxNcC2M77mw9W3cCHbUHy'\ndf = pd.read_csv(file_path)\n\n# We only need the 'Date' and 'Open' columns for the JSON output\ndf_json = df[['Date', 'Open']]\n\n# Convert the 'Date' format to 'YYYY-MM-DD' without time\ndf_json['Date'] = pd.to_datetime(df_json['Date']).dt.strftime('%Y-%m-%d')\n\n# Convert the DataFrame to a list of dictionaries\njson_output = df_json.to_dict(orient='records')\n\n# Convert list of dictionaries to JSON format\njson_data = json.dumps(json_output, ensure_ascii=False, indent=2)\n\n# Save the JSON data to a file\njson_file_path = '/mnt/data/stock_data.json'\nw

In [76]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

もちろんです。以下に、アップロードされたCSVデータを読み込み、JSON形式に変換し、ファイルに保存するために使用したPythonコードを記載します。

```python
import pandas as pd
import json

# Load the uploaded CSV file
file_path = '/mnt/data/assistant-DffPxNcC2M77mw9W3cCHbUHy'
df = pd.read_csv(file_path)

# We only need the 'Date' and 'Open' columns for the JSON output
df_json = df[['Date', 'Open']]

# Convert the 'Date' format to 'YYYY-MM-DD' without time
df_json['Date'] = pd.to_datetime(df_json['Date']).dt.strftime('%Y-%m-%d')

# Convert the DataFrame to a list of dictionaries
json_output = df_json.to_dict(orient='records')

# Convert list of dictionaries to JSON format
json_data = json.dumps(json_output, ensure_ascii=False, indent=2)

# Save the JSON data to a file
json_file_path = '/mnt/data/stock_data.json'
with open(json_file_path, 'w', encoding='utf-8') as file:
    file.write(json_data)

# Provide the file path for download
json_file_path
```

このコードは、まず `pandas` ライブラリを使用してCSVファイルを読み込み、必要な列のみを選択し、`strftime` 関数で日付形式を変換します。その後、`to_dict` メソッドでデータフレーム

##### タスク1 - 発展：EM アルゴリズムを解く

In [4]:
request = """
ふたつのコインA, Bがあります。コイントスをすると、AのほうがBより表が出やすいことが分かっています。どちらかのコインを使って20回コイントスを行うという試行を10回繰り返した結果、表が出た回数は次のようになりました。A、Bそれぞれのコインの表が出る確率をEMアルゴリズムで推定してください。
表が出た回数	15	4	6	13	3	5	4	4	7	2
"""

In [5]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model"
)

In [6]:
thread = client.beta.threads.create()

In [7]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 3 minutes 56 seconds
{
  "data": [
    {
      "id": "msg_0035aBecFiZRkXqyLqZuJtQ1",
      "assistant_id": "asst_4BKrPWfAgZxbfumpy5n9fMOi",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "EMアルゴリズムに基づく推定結果によると、コインAの表が出る確率は約0.2185（約21.85%）、コインBの表が出る確率は約0.6970（約69.70%）であることが示されました。\n\nEMアルゴリズムの性質上、初期値によって最終結果が若干異なることがあります。したがって、この推定結果はその実行の一例であり、異なるランダムシードを使用すると異なる推定が得られる可能性があります。ただし、推定された確率がAの方がBよりも明らかに小さいことから、初期の記述に合致していることがわかります。\n\n対数尤度の最終値は約-25.279です。この尤度は推定されたモデルがデータをどれくらいよく説明しているかの指標であり、EMアルゴリズムはこの尤度を最大化するようなパラメータを探索します。尤度が最大となる地点（または収束判定基準に十分近い地点）でアルゴリズムが停止します。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708496892,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_l2xoDRaL4YybLvO7yamnPWd8",
      "thread_id": "thread_wLveoOoQremQTeAyu3xTD0rH"
    },
    {
      "id": "msg_a6eTDQkAG3MaaDP

In [8]:
get_response(client=client, assistant=assistant, thread=thread, request="使用したコードを教えてください")

Status: completed
Elapsed time: 1 minutes 54 seconds
{
  "data": [
    {
      "id": "msg_uXTQfprFBaKjm5P4dogOUA6w",
      "assistant_id": "asst_4BKrPWfAgZxbfumpy5n9fMOi",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下はコインAとコインBの表が出る確率を推定するために使用したEMアルゴリズムのコードです。このコードはコイントスの結果を受け取り、両コインの確率を推定します。\n\n```python\nimport numpy as np\nfrom scipy.stats import binom\n\ndef em_algorithm(tosses, tol=1e-6, max_iter=10000):\n    # 初期パラメータpの推定をランダムにセット\n    p = np.array([np.random.rand(), np.random.rand()])\n    \n    # 変数の初期化\n    n = len(tosses)  # トスの回数\n    heads = np.array(tosses)  # 表が出た回数\n    num_tosses = 20  # それぞれの試行でのトスの回数\n    \n    log_likelihood = -np.inf\n    \n    # EMアルゴリズムの実行\n    for _ in range(max_iter):\n        # E-step: 各トライアルのコインA、Bからそれぞれのデータが来たとする確率の割合を計算\n        w_a = binom.pmf(heads, num_tosses, p[0]) * 0.5  # コインAによる寄与(等確率を仮定)\n        w_b = binom.pmf(heads, num_tosses, p[1]) * 0.5  # コインBによる寄与(等確率を仮定)\n  

In [9]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

{
  "data": [
    {
      "id": "msg_uXTQfprFBaKjm5P4dogOUA6w",
      "assistant_id": "asst_4BKrPWfAgZxbfumpy5n9fMOi",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下はコインAとコインBの表が出る確率を推定するために使用したEMアルゴリズムのコードです。このコードはコイントスの結果を受け取り、両コインの確率を推定します。\n\n```python\nimport numpy as np\nfrom scipy.stats import binom\n\ndef em_algorithm(tosses, tol=1e-6, max_iter=10000):\n    # 初期パラメータpの推定をランダムにセット\n    p = np.array([np.random.rand(), np.random.rand()])\n    \n    # 変数の初期化\n    n = len(tosses)  # トスの回数\n    heads = np.array(tosses)  # 表が出た回数\n    num_tosses = 20  # それぞれの試行でのトスの回数\n    \n    log_likelihood = -np.inf\n    \n    # EMアルゴリズムの実行\n    for _ in range(max_iter):\n        # E-step: 各トライアルのコインA、Bからそれぞれのデータが来たとする確率の割合を計算\n        w_a = binom.pmf(heads, num_tosses, p[0]) * 0.5  # コインAによる寄与(等確率を仮定)\n        w_b = binom.pmf(heads, num_tosses, p[1]) * 0.5  # コインBによる寄与(等確率を仮定)\n        w_total = w_a + w_b\n        w_a /= w_total\n  

In [10]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

もちろんです。以下はコインAとコインBの表が出る確率を推定するために使用したEMアルゴリズムのコードです。このコードはコイントスの結果を受け取り、両コインの確率を推定します。

```python
import numpy as np
from scipy.stats import binom

def em_algorithm(tosses, tol=1e-6, max_iter=10000):
    # 初期パラメータpの推定をランダムにセット
    p = np.array([np.random.rand(), np.random.rand()])
    
    # 変数の初期化
    n = len(tosses)  # トスの回数
    heads = np.array(tosses)  # 表が出た回数
    num_tosses = 20  # それぞれの試行でのトスの回数
    
    log_likelihood = -np.inf
    
    # EMアルゴリズムの実行
    for _ in range(max_iter):
        # E-step: 各トライアルのコインA、Bからそれぞれのデータが来たとする確率の割合を計算
        w_a = binom.pmf(heads, num_tosses, p[0]) * 0.5  # コインAによる寄与(等確率を仮定)
        w_b = binom.pmf(heads, num_tosses, p[1]) * 0.5  # コインBによる寄与(等確率を仮定)
        w_total = w_a + w_b
        w_a /= w_total
        w_b /= w_total
        
        # M-step: 新しいパラメータの推定値を計算
        p_new = np.array([
            np.sum(w_a * heads) / np.sum(w_a * num_tosses),
            np.sum(w_b * heads) / np.sum(w_b * num_tosses)
        ])
        
        # 対数尤度の

##### タスク2 - 発展：時系列予測結果を可視化する

In [11]:
request = "添付データを元に、2019年までのOpenの値から予測モデルを構築し、予測した2020年以降のOpenの値を実際の値と合わせてグラフに表示してください"

In [12]:
file = client.files.create(
    file=open("Microsoft_Stock.csv", "rb"),
    purpose='assistants'
)

In [13]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model",
    file_ids=[file.id]
)

In [14]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_4FCewxolkWnxhqkci14Jx0ly', created_at=1708498111, metadata={}, object='thread')


In [15]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 2 minutes 22 seconds
{
  "data": [
    {
      "id": "msg_ssiM4JE4D1ZR7YKvKRMGyuKQ",
      "assistant_id": "asst_aqv7LILmkJt9ZQU8KaALQ4UO",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-86UzzaDx4CnLndC4si3mhTTz"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "グラフには、2019年までの実際の「Open」の値（青色）、2020年以降の実際の「Open」の値（緑色）、そして予測された「Open」の値（赤色、破線）が表示されています。線形回帰モデルを用いると、長期的な傾向を捉えることができますが、短期的な変動や特定のパターンは捉えられません。\n\nこのモデルは単純なので、実際のデータの複雑な挙動に完全には対応していません。より詳細な分析や予測を行うためには、時系列分析専用のモデルやディープラーニングアプローチを用いることが考えられます。もし必要であれば、より洗練された時系列モデルを構築する手助けもできます。ご希望であればお知らせください。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708498229,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_oEbTr086B2gy6Y6Q0xpOpaVS",
      "thread_id": "thread_4FCewxolkW

In [19]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

{
  "data": [
    {
      "id": "msg_ssiM4JE4D1ZR7YKvKRMGyuKQ",
      "assistant_id": "asst_aqv7LILmkJt9ZQU8KaALQ4UO",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-86UzzaDx4CnLndC4si3mhTTz"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "グラフには、2019年までの実際の「Open」の値（青色）、2020年以降の実際の「Open」の値（緑色）、そして予測された「Open」の値（赤色、破線）が表示されています。線形回帰モデルを用いると、長期的な傾向を捉えることができますが、短期的な変動や特定のパターンは捉えられません。\n\nこのモデルは単純なので、実際のデータの複雑な挙動に完全には対応していません。より詳細な分析や予測を行うためには、時系列分析専用のモデルやディープラーニングアプローチを用いることが考えられます。もし必要であれば、より洗練された時系列モデルを構築する手助けもできます。ご希望であればお知らせください。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708498229,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_oEbTr086B2gy6Y6Q0xpOpaVS",
      "thread_id": "thread_4FCewxolkWnxhqkci14Jx0ly"
    },
    {
      "id": "msg_UupjlWZ

In [20]:
data = json.loads(messages.model_dump_json(indent=2))
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)

assistant-86UzzaDx4CnLndC4si3mhTTz


In [21]:
content = client.files.content(image_file_id)

image = content.write_to_file("image.png")

In [22]:
from PIL import Image

# Display the image in the default image viewer
image = Image.open("image.png")
image.show()

In [23]:
get_response(client=client, assistant=assistant, thread=thread, request="使用したコードを教えてください")

Status: completed
Elapsed time: 1 minutes 4 seconds
{
  "data": [
    {
      "id": "msg_irAg9ldhvWV6PNdRBBaIqIJu",
      "assistant_id": "asst_aqv7LILmkJt9ZQU8KaALQ4UO",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下は、アップロードされたデータを読み込んで線形回帰モデルを構築し、2019年までの「Open」の値から予測を行い、その結果をグラフに表示するために使用したコードです。\n\n```python\nimport pandas as pd\nfrom sklearn.linear_model import LinearRegression\nimport matplotlib.pyplot as plt\nimport numpy as np\n\n# Load the uploaded file\nfile_path = '/mnt/data/assistant-EmOKijP71DrqLQJI2mi5v1RA'\ndata = pd.read_csv(file_path)\n\n# Convert 'Date' to datetime\ndata['Date'] = pd.to_datetime(data['Date'])\n\n# Sort data by date just in case\ndata.sort_values('Date', inplace=True)\n\n# Filter data up to the end of 2019\ndata_train = data[data['Date'].dt.year <= 2019]\n\n# Prepare training data\nX_train = np.array(range(len(data_train))).reshape(-1, 1)  # We use days as numbers for simplicity\ny_train =

In [24]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

{
  "data": [
    {
      "id": "msg_irAg9ldhvWV6PNdRBBaIqIJu",
      "assistant_id": "asst_aqv7LILmkJt9ZQU8KaALQ4UO",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下は、アップロードされたデータを読み込んで線形回帰モデルを構築し、2019年までの「Open」の値から予測を行い、その結果をグラフに表示するために使用したコードです。\n\n```python\nimport pandas as pd\nfrom sklearn.linear_model import LinearRegression\nimport matplotlib.pyplot as plt\nimport numpy as np\n\n# Load the uploaded file\nfile_path = '/mnt/data/assistant-EmOKijP71DrqLQJI2mi5v1RA'\ndata = pd.read_csv(file_path)\n\n# Convert 'Date' to datetime\ndata['Date'] = pd.to_datetime(data['Date'])\n\n# Sort data by date just in case\ndata.sort_values('Date', inplace=True)\n\n# Filter data up to the end of 2019\ndata_train = data[data['Date'].dt.year <= 2019]\n\n# Prepare training data\nX_train = np.array(range(len(data_train))).reshape(-1, 1)  # We use days as numbers for simplicity\ny_train = data_train['Open'].values\n\n# Create and train the

In [25]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

もちろんです。以下は、アップロードされたデータを読み込んで線形回帰モデルを構築し、2019年までの「Open」の値から予測を行い、その結果をグラフに表示するために使用したコードです。

```python
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np

# Load the uploaded file
file_path = '/mnt/data/assistant-EmOKijP71DrqLQJI2mi5v1RA'
data = pd.read_csv(file_path)

# Convert 'Date' to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Sort data by date just in case
data.sort_values('Date', inplace=True)

# Filter data up to the end of 2019
data_train = data[data['Date'].dt.year <= 2019]

# Prepare training data
X_train = np.array(range(len(data_train))).reshape(-1, 1)  # We use days as numbers for simplicity
y_train = data_train['Open'].values

# Create and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict the Open values using the model
data['Predicted_Open'] = model.predict(np.array(range(len(data))).reshape(-1, 1))

# Split the dataframe into training+vali

##### タスク3 - 発展：CSV を月毎に平均を取ってから JSON にする

In [26]:
request = """
アップロードしたCSVを次のような月毎に平均を取ったJSONに変換して出力してください
# 出力JSONの例
[
  {"Date": "2015-4", "Open": 40.6},
  {"Date": "2015-5", "Open": 40.66},
  …
]
"""

In [27]:
file = client.files.create(
    file=open("Microsoft_Stock.csv", "rb"),
    purpose='assistants'
)

In [28]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model",
    file_ids=[file.id]
)

In [29]:
thread = client.beta.threads.create()

In [30]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 1 minutes 27 seconds
{
  "data": [
    {
      "id": "msg_3Be65NWza3MjgsNKbMH5TSEm",
      "assistant_id": "asst_vLzqB7KJARK2SZm0xlOXFqlM",
      "content": [
        {
          "text": {
            "annotations": [
              {
                "end_index": 141,
                "file_path": {
                  "file_id": "assistant-Gk7jMzrWVcrg8gSTn0mXTkkV"
                },
                "start_index": 95,
                "text": "sandbox:/mnt/data/monthly_avg_open_prices.json",
                "type": "file_path"
              }
            ],
            "value": "月毎の「Open」平均値を含むJSONデータファイルが生成されました。以下のリンクからダウンロードできます：\n\n[ダウンロード: monthly_avg_open_prices.json](sandbox:/mnt/data/monthly_avg_open_prices.json)"
          },
          "type": "text"
        }
      ],
      "created_at": 1708499255,
      "file_ids": [
        "assistant-Gk7jMzrWVcrg8gSTn0mXTkkV"
      ],
      "metadata": {},
      "object": "thread.message",
      "role": "assist

In [31]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

{
  "data": [
    {
      "id": "msg_3Be65NWza3MjgsNKbMH5TSEm",
      "assistant_id": "asst_vLzqB7KJARK2SZm0xlOXFqlM",
      "content": [
        {
          "text": {
            "annotations": [
              {
                "end_index": 141,
                "file_path": {
                  "file_id": "assistant-Gk7jMzrWVcrg8gSTn0mXTkkV"
                },
                "start_index": 95,
                "text": "sandbox:/mnt/data/monthly_avg_open_prices.json",
                "type": "file_path"
              }
            ],
            "value": "月毎の「Open」平均値を含むJSONデータファイルが生成されました。以下のリンクからダウンロードできます：\n\n[ダウンロード: monthly_avg_open_prices.json](sandbox:/mnt/data/monthly_avg_open_prices.json)"
          },
          "type": "text"
        }
      ],
      "created_at": 1708499255,
      "file_ids": [
        "assistant-Gk7jMzrWVcrg8gSTn0mXTkkV"
      ],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_1Sy91HKPVFiVK2yRF2JhZaMY",

In [32]:
data = json.loads(messages.model_dump_json(indent=2))
file_id = data["data"][0]["content"][0]["text"]["annotations"][0]["file_path"]["file_id"]

In [33]:
content = client.files.content(file_id)
content_bytes = content.read()

with open("./task3_progress.json", "wb") as file:
    file.write(content_bytes)

##### できないこと

In [5]:
request = "IBMの株価を1991年から2023年までmatplotlibでplotしてください"

In [6]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model"
)

In [7]:
thread = client.beta.threads.create()

In [8]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 0 minutes 22 seconds
{
  "data": [
    {
      "id": "msg_at0qYL1jmDaz5q4fG1EHoYqS",
      "assistant_id": "asst_JgGHGeM3NGhOHygUh4tPQJ3n",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "申し訳ありませんが、この環境にはインターネットアクセスが制限されているため、直接株の価格データを取得することはできません。ただし、データを持っている場合は、それをアップロードしてからグラフを作成できます。お持ちのデータをアップロードして指示をしていただければ、それに基づいてIBMの株価をプロットすることができます。データがCSVやExcelファイルであれば、それを解析してグラフにすることが可能です。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708580299,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_lnb2C6St2Wisry4hbK9COEhg",
      "thread_id": "thread_OCZ0MZDJdL1pdyBisUy2UetC"
    },
    {
      "id": "msg_O4NCSc0j4BgMIcqAaevXSuhC",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "IBMの株価を1991年から2023年までmatplotlibでplotしてくださ

##### 資料用に正規分布のグラフを作成する

###### 正規分布

In [5]:
request = "正規分布のグラフを表示してください。プロット以外の情報は全て消してください。"

In [6]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model"
)

In [7]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_3qO4OFcBYjIWfyboc4utS5JT', created_at=1709008374, metadata={}, object='thread')


In [8]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 0 minutes 38 seconds
{
  "data": [
    {
      "id": "msg_ft3LJsAAslJqlpydeuUGF6lX",
      "assistant_id": "asst_1GCrrlv0xt9H32YmGsmLN3at",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-8otPoMqMDxdNEXfDGTQjOGXD"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the graph of a normal distribution with all plot information removed, as requested."
          },
          "type": "text"
        }
      ],
      "created_at": 1709008414,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_dSnXE3kIynByykOrJzrt1PUs",
      "thread_id": "thread_3qO4OFcBYjIWfyboc4utS5JT"
    },
    {
      "id": "msg_nsrXOxlTFhjfeAgAOtiaZC5Y",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "

In [9]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_ft3LJsAAslJqlpydeuUGF6lX",
      "assistant_id": "asst_1GCrrlv0xt9H32YmGsmLN3at",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-8otPoMqMDxdNEXfDGTQjOGXD"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the graph of a normal distribution with all plot information removed, as requested."
          },
          "type": "text"
        }
      ],
      "created_at": 1709008414,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_dSnXE3kIynByykOrJzrt1PUs",
      "thread_id": "thread_3qO4OFcBYjIWfyboc4utS5JT"
    },
    {
      "id": "msg_nsrXOxlTFhjfeAgAOtiaZC5Y",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "正規分布のグラフを表示してください。プロット以外の情報は全て消してください。"
          },


In [10]:
data = json.loads(messages.model_dump_json(indent=2))
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)

assistant-8otPoMqMDxdNEXfDGTQjOGXD


In [11]:
content = client.files.content(image_file_id)

image = content.write_to_file("image.png")

###### 混合正規分布

In [12]:
request = "混合正規分布のグラフを表示してください。プロット以外の情報は全て消してください。"

In [13]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 0 minutes 35 seconds
{
  "data": [
    {
      "id": "msg_1JyDHoRvvSges1YyaEA5auWT",
      "assistant_id": "asst_1GCrrlv0xt9H32YmGsmLN3at",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-Fr8BdJw6Gt0rITPpw1iw9YeJ"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the graph of a mixed normal distribution with all plot information removed, as requested."
          },
          "type": "text"
        }
      ],
      "created_at": 1709008629,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_qkS9UDW2fYHyW2IiRhcjd8Ek",
      "thread_id": "thread_3qO4OFcBYjIWfyboc4utS5JT"
    },
    {
      "id": "msg_EEnq1b6TIvpvXHhjTFR9aonc",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "val

In [14]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

In [15]:
data = json.loads(messages.model_dump_json(indent=2))
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)

assistant-Fr8BdJw6Gt0rITPpw1iw9YeJ


In [16]:
content = client.files.content(image_file_id)

image = content.write_to_file("image.png")